# Billboard Music Awards

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
#2014

def extract_year_from_url(url):
    match = re.search(r'(\d{4})_', url)
    if match:
        year = match.group(1)
        return year
    else:
        return None

def scrape_table_1(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tables = soup.find_all('table')
        if len(tables) >= 4:
            second_table = tables[4]
            data = []
            rows = second_table.find_all('tr')
            for i in range(len(rows)):
                row_text = rows[i].get_text().strip().split('\n')
                if i % 2 == 0:
                    categories = [category.strip() for category in row_text if category.strip()]
                else:
                    cells = rows[i].find_all(['td', 'th'])
                    winners = []
                    for cell in cells:
                        winner_list = cell.find_all('ul')
                        if winner_list:
                            winner_name = winner_list[0].find('li').get_text().strip().split('\n')[0]
                            winners.append(winner_name)
                        else:
                            winners.append(cell.get_text().strip())  
                    for idx, category in enumerate(categories):
                        if idx < len(winners):
                            winner = winners[idx]
                        else:
                            winner = ''
                        data.append({'Category': category, 'Winner': winner})
            df = pd.DataFrame(data)
            return df
        else:
            print('Non è stata trovata una quarta tabella sulla pagina.')
            return None
    else:
        print('Errore nella richiesta HTTP:', response.status_code)
        return None

url = "https://en.wikipedia.org/wiki/2014_Billboard_Music_Awards"

result_1 = scrape_table_1(url)
result_1['Year']=extract_year_from_url(url)

In [3]:
#2013-2015-2016-2017-2018-2019-2020-2021-2022-2023

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def extract_year_from_url(url):
    match = re.search(r'(\d{4})_', url)
    if match:
        year = match.group(1)
        return year
    else:
        return None

def scrape_table_2(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tables = soup.find_all('table')
        if len(tables) >= 4:
            second_table = tables[3]
            data = []
            rows = second_table.find_all('tr')
            for i in range(len(rows)):
                row_text = rows[i].get_text().strip().split('\n')
                if i % 2 == 0:
                    categories = [category.strip() for category in row_text if category.strip()]
                else:
                    cells = rows[i].find_all(['td', 'th'])
                    winners = []
                    for cell in cells:
                        winner_list = cell.find_all('ul')
                        if winner_list:
                            winner_name = winner_list[0].find('li').get_text().strip().split('\n')[0]
                            winners.append(winner_name)
                        else:
                            winners.append(cell.get_text().strip())  
                    for idx, category in enumerate(categories):
                        if idx < len(winners):
                            winner = winners[idx]
                        else:
                            winner = ''
                        data.append({'Category': category, 'Winner': winner})
            df = pd.DataFrame(data)
            return df
        else:
            print('Non è stata trovata una quarta tabella sulla pagina.')
            return None
    else:
        print('Errore nella richiesta HTTP:', response.status_code)
        return None


urls = [
    'https://en.wikipedia.org/wiki/2013_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2015_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2016_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2017_Billboard_Music_Awards', 
    'https://en.wikipedia.org/wiki/2018_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2019_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2020_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2021_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2022_Billboard_Music_Awards',
    'https://en.wikipedia.org/wiki/2023_Billboard_Music_Awards',  
]

all_dfs = []

for url in urls:
    year = extract_year_from_url(url)
    df_results = scrape_table_2(url)
    if df_results is not None:
        df_results['Year'] = year
        all_dfs.append(df_results)

result_2 = pd.concat(all_dfs, ignore_index=True)

In [5]:
BillboardMusicAwards = pd.concat([result_1,result_2])

In [6]:
BillboardMusicAwards['Competition'] = 'BillboardMusicAwards'

Select the song

In [7]:
import re

def extract_text_in_quotes(text):
    match = re.search(r'"([^"]*)"', text)
    if match:
        return match.group(1)
    else:
        return None

BillboardMusicAwards['Song'] = BillboardMusicAwards['Winner'].apply(extract_text_in_quotes)

Select the album

In [8]:
# dal 2013 al 2019 l'album è prima del –

In [9]:
def extract_album_1(text):
    match = re.search(r'(?<!")[^"–]+(?!\s*")\s*–', text)
    if match:
        return match.group(0).strip()[:-1]  
    else:
        return None

In [10]:
# dal 2019 al 2023 l'album è dopo il –

In [11]:
def extract_album_2(text):
    match = re.search(r'–\s*([^"–]+|$)', text)
    if match:
        album = match.group(1).strip()
        if album.startswith('"') and album.endswith('"') or not album:
            return None
        else:
            return album
    else:
        return None

In [12]:
BillboardMusicAwards['Year'] = pd.to_numeric(BillboardMusicAwards['Year'], errors='coerce') #convert in int

In [13]:
BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2020) & (BillboardMusicAwards['Year'] <= 2023), 'Album'] = BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2020) & (BillboardMusicAwards['Year'] <= 2023), 'Winner'].apply(extract_album_2)
BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2013) & (BillboardMusicAwards['Year'] <= 2019), 'Album'] = BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2013) & (BillboardMusicAwards['Year'] <= 2019), 'Winner'].apply(extract_album_1)

Select artist

In [14]:
def extract_remaining_text_1(text):
    if '"' in text and ('-' in text or '–' in text):
        match = re.search(r'"([^"]+)"\s*[-–]\s*([^–]+)', text)
        if match:
            return match.group(2)
    elif '-' in text or '–' in text:
        parts = re.split(r'[-–]', text)
        return parts[-1].strip()
    elif ',' in text:
        return text.split(',')[0].strip()  
    elif '"' in text:
        match = re.search(r'"([^"]+)"', text)
        if match:
            return match.group(1)
    else:
        return text

In [15]:
def extract_remaining_text_2(text):
    if '"' in text:
        if '-' in text or '–' in text:
            return text.split('-')[0].split('–')[0].strip()
        else:
            return text.strip('"')
    elif '-' in text or '–' in text:
        return text.split('-')[0].split('–')[0].strip()
    else:
        return text.strip()

In [16]:
BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2013) & (BillboardMusicAwards['Year'] <= 2019), 'Remaining_Text'] = BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2013) & (BillboardMusicAwards['Year'] <= 2019), 'Winner'].apply(extract_remaining_text_1)
BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2020) & (BillboardMusicAwards['Year'] <= 2023), 'Remaining_Text'] = BillboardMusicAwards.loc[(BillboardMusicAwards['Year'] >= 2020) & (BillboardMusicAwards['Year'] <= 2023), 'Winner'].apply(extract_remaining_text_2)

Split featuring e artist

In [17]:
def split_remaining_text(remaining_text):
    if remaining_text:
        if "featuring" in remaining_text.lower() or "feat" in remaining_text.lower() or "ft." in remaining_text.lower():
            featuring_index = remaining_text.lower().find("featuring")
            feat_index = remaining_text.lower().find("feat")
            ft_index = remaining_text.lower().find("ft.")
            
            feat_start_index = max(feat_index, featuring_index,ft_index)
            featuring_text = remaining_text[feat_start_index:].replace("featuring", "").replace("feat", "").replace("ft.", "").strip()
            
            artist_text = remaining_text[:feat_start_index].strip()
        else:
            featuring_text = None
            artist_text = remaining_text.strip()
    else:
        featuring_text = None
        artist_text = None

    return featuring_text, artist_text

BillboardMusicAwards['Featuring'], BillboardMusicAwards['Artist'] = zip(*BillboardMusicAwards['Remaining_Text'].apply(split_remaining_text))

In [18]:
def clean_featuring(text):
    if text is None:
        return None
    cleaned_text = re.sub(r'\(.*?\)', '', text)  
    cleaned_text = re.sub(r'\).*$', '', cleaned_text)  
    cleaned_text = re.sub(r'feat\.', '', cleaned_text)  
    cleaned_text = re.sub(r'^\.', '', cleaned_text)  
    return cleaned_text.strip()  

BillboardMusicAwards['Featuring'] = BillboardMusicAwards['Featuring'].apply(clean_featuring)

In [19]:
def clean_artist(text):
    if text is None:
        return None
    cleaned_text = text.replace('(', '')  
    cleaned_text = cleaned_text.replace(')', '')  
    return cleaned_text

BillboardMusicAwards['Artist'] = BillboardMusicAwards['Artist'].apply(clean_artist)

In [20]:
BillboardMusicAwards.drop(columns=['Remaining_Text'], inplace=True)

Split the artists

In [21]:
BillboardMusicAwards_tosplit = BillboardMusicAwards[BillboardMusicAwards['Year'] != 2014]

In [22]:
def split_artists(df, column_name):
    artist_count = 1
    
    for index, row in df.iterrows():
        if pd.notna(row[column_name]):
            artists = re.split(r'[,&]|(?:\sand\s)|\swith\s', row[column_name])
            for i, artist in enumerate(artists, start=1):
                df.at[index, f'Artist {i}'] = artist.strip()
    df.drop(columns=[column_name], inplace=True)
    
    return df


BillboardMusicAwards_splitted = split_artists(BillboardMusicAwards_tosplit, 'Artist')

C:\Users\fspor\AppData\Local\Temp\ipykernel_23120\3977235794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f'Artist {i}'] = artist.strip()
C:\Users\fspor\AppData\Local\Temp\ipykernel_23120\3977235794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f'Artist {i}'] = artist.strip()
C:\Users\fspor\AppData\Local\Temp\ipykernel_23120\3977235794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [23]:
BillboardMusicAwards_2014 = BillboardMusicAwards[BillboardMusicAwards['Year'] == 2014]

In [24]:
BillboardMusicAwards_2014 = BillboardMusicAwards_2014.rename(columns={'Artist': 'Artist 1'})

In [25]:
BillboardMusicAwards_new = pd.concat([BillboardMusicAwards_2014,BillboardMusicAwards_splitted])

In [26]:
# Manual row corrections
BillboardMusicAwards_new.loc[153, 'Artist 1'] = 'BTS'
BillboardMusicAwards_new.loc[467, 'Artist 1'] = 'Silk Sonic'
BillboardMusicAwards_new.loc[148, 'Artist 1'] = 'The Chainsmokers'
BillboardMusicAwards_new.loc[148, 'Featuring'] = 'Halsey'
BillboardMusicAwards_new.loc[163, 'Artist 1'] = 'Kenny Chesney'
BillboardMusicAwards_new.loc[163, 'Featuring'] = 'Pink'
BillboardMusicAwards_new.loc[174, 'Artist 1'] = 'Drake'
BillboardMusicAwards_new.loc[174, 'Featuring'] = 'WizKid & Kyla'
BillboardMusicAwards_new.loc[179, 'Artist 1'] = 'Rae Sremmurd'
BillboardMusicAwards_new.loc[179, 'Featuring'] = 'Gucci Mane'
BillboardMusicAwards_new.loc[204, 'Artist 1'] = 'Luis Fonsi'
BillboardMusicAwards_new.loc[204, 'Artist 2'] = 'Daddy Yankee'
BillboardMusicAwards_new.loc[204, 'Featuring'] = 'Justin Bieber'
BillboardMusicAwards_new.loc[354, 'Artist 1'] = 'Ellie Goulding'
BillboardMusicAwards_new.loc[354, 'Artist 2'] = 'Diplo'

In [27]:
BillboardMusicAwards_new['Category'] = BillboardMusicAwards_new['Category'].replace('Milestone Award (fan-voted)[6]', 'Milestone Award (fan-voted)')

In [28]:
categorie_da_nullificare = [
    "Milestone Award (fan-voted)",
    "Top Country Tour",
    "Top R&B Tour",
    "Top Rap Tour",
    "Top Rock Tour",
    "Top Soundtrack/Cast Album",
    "Top Soundtrack",
    "Top Tour",
    "Top Latin Tour"
]

for categoria in categorie_da_nullificare:
    rows_to_nullify = BillboardMusicAwards_new[BillboardMusicAwards_new['Category'] == categoria]
    
    for index, row in rows_to_nullify.iterrows():
        for colonna in BillboardMusicAwards_new.columns:
            if colonna not in ['Category', 'Winner', 'Year', 'Competition']:
                BillboardMusicAwards_new.at[index, colonna] = None

In [29]:
new_order = ['Category', 'Winner', 'Year', 'Competition', 'Song', 'Album', 'Artist 1', 'Artist 2', 'Artist 3', 'Artist 4', 'Artist 5', 'Artist 6', 'Featuring']

BillboardMusicAwards_new = BillboardMusicAwards_new.reindex(columns=new_order)

### Save as csv file

In [30]:
BillboardMusicAwards_new.to_csv('BMA_final.csv')

# Analysis

### Who is the most successful Artist?

In [31]:
from collections import Counter

artists_list = BillboardMusicAwards_new[['Artist 1', 'Artist 2', 'Artist 3','Artist 4', 'Artist 5', 'Artist 6']].values.ravel()
artists_list = [artist for artist in artists_list if pd.notnull(artist)]  

artist_counter = Counter(artists_list)
most_successful_artist, most_successful_artist_count = artist_counter.most_common(1)[0]

print("The most successful Artist is:", most_successful_artist)
print("Number of Awards:", most_successful_artist_count)

The most successful Artist is: Drake
Number of Awards: 38


### Which is the most awarded Album?

In [32]:
album_counts = BillboardMusicAwards_new['Album'].value_counts()
max_occurrences = album_counts.max()
most_awarded_albums = album_counts[album_counts == max_occurrences].index.tolist()

print("The most awarded Albums are:", most_awarded_albums)
print("Number of Awards:", max_occurrences)

The most awarded Albums are: ['The 20/20 Experience ', 'One Thing at a Time', 'DAMN. ', 'Views ', 'Jesus Is King', 'Shoot for the Stars, Aim for the Moon', 'Traveller ', 'Scorpion ', 'Donda', 'How Can It Be ', 'Red ']
Number of Awards: 2


### Which is the most awarded song?

In [33]:
song_counts = BillboardMusicAwards_new['Song'].value_counts()
max_occurrences = song_counts.max()
most_awarded_songs = song_counts[song_counts == max_occurrences].index.tolist()

print("The most awarded Songs are:", most_awarded_songs)
print("Number of Awards:", max_occurrences)

The most awarded Songs are: ['Despacito', 'Stay']
Number of Awards: 5


### Which artist has made the greatest contribution to the victory?

In [34]:
featuring_counts = BillboardMusicAwards_new['Featuring'].value_counts()
max_occurrences = featuring_counts.max()
most_frequent_featurings = featuring_counts[featuring_counts == max_occurrences].index.tolist()

print("The Artist who did the highest number of Featuring collaborations is:", most_frequent_featurings)
print("Number of Awards:", max_occurrences)


The Artist who did the highest number of Featuring collaborations is: ['Justin Bieber']
Number of Awards: 5


### In which year were the categories most inclusive?

In [35]:
year_counts = BillboardMusicAwards_new['Year'].value_counts()
max_occurrences = year_counts.max()
most_inclusive_years = year_counts[year_counts == max_occurrences].index.tolist()

print("The most inclusive Year is:", most_inclusive_years)
print("Number of Categories:", max_occurrences)



The most inclusive Year is: [2023]
Number of Categories: 69
